In [31]:
import subprocess
import folium
from geopy.geocoders import Nominatim
import requests

def trace_route(destination):
    """Функция для выполнения traceroute и получения траектории сетевых маршрутов.
    :param destination: Конечный сервер (IP или доменное имя)
    :return: Список хопов с временем ответа и их IP-адресами
    """
    command = ['tracert', '-h', '30', '-w', '200', '-d', destination]  # Для Windows
    result = subprocess.check_output(command, universal_newlines=True)
    return result

def get_geolocation(ip):
    # url = f"http://ip-api.com/json/{ip}"
    url = f"http://ipwho.is/{ip}"
    
    response = requests.get(url)
    data = response.json()
    # if data['status'] == 'success':
    #     return data['lat'], data['lon']
    if data['success'] == True:
        return data['latitude'], data['longitude']
    return None, None



def plot_traceroute_on_map(trace_output):
    """
    Функция для отображения результатов трассировки на карте.
    :param trace_output: Вывод команды traceroute
    """
    ips = []
    for line in trace_output.splitlines():
        if 'ms' in line:
            parts = line.split()
            ip = parts[len(parts)-1] if len(parts) > 2 else None
            if ip:
                ip = ip.replace("[", "").replace("]", "")
                ips.append(ip)

    # Инициализация карты с базовыми координатами
    m = folium.Map(location=[20, 0], zoom_start=2)
    
    latitudes = []
    longitudes = []
    coordinates = []

    # Добавляем маркеры на карту
    for ip in ips:
        lat, lon = get_geolocation(ip)
        if lat and lon:
            # Сохраняем координаты
            coordinates.append((lat, lon))
            
            # Добавляем круговой маркер
            folium.CircleMarker(
                location=(lat, lon),
                radius=5,
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.6,
                popup=f'IP: {ip}'
            ).add_to(m)
    
    # Соединяем точки линиями
    if coordinates:
        folium.PolyLine(
            locations=coordinates,
            color="red",
            weight=2.5,
            opacity=0.8
        ).add_to(m)

    # Сохраняем карту
    m.save("traceroute_map.html")
    print("Карта с трассировкой сохранена как 'traceroute_map.html'")

    return coordinates



In [32]:
# Пример использования
destination = "vk.com"  # Вы можете заменить на любой сервер
trace_output = trace_route(destination)
coord = plot_traceroute_on_map(trace_output)

Карта с трассировкой сохранена как 'traceroute_map.html'


In [34]:
from geopy.distance import geodesic


def calculate_distances(coords):
    distances = []
    for i in range(len(coords) - 1):
        distance = geodesic(coords[i], coords[i + 1]).kilometers
        distances.append(distance)
    return distances

distances = calculate_distances(coord)
for i, distance in enumerate(distances):
    print(f"Distance between point {i + 1} and point {i + 2}: {distance:.2f} km")


Distance between point 1 and point 2: 0.00 km
Distance between point 2 and point 3: 0.00 km
Distance between point 3 and point 4: 1326.56 km
Distance between point 4 and point 5: 2966.15 km
Distance between point 5 and point 6: 9876.42 km
Distance between point 6 and point 7: 9412.21 km
Distance between point 7 and point 8: 1701.41 km
Distance between point 8 and point 9: 818.14 km
Distance between point 9 and point 10: 599.25 km
Distance between point 10 and point 11: 1188.61 km
Distance between point 11 and point 12: 694.33 km
Distance between point 12 and point 13: 1913.36 km
Distance between point 13 and point 14: 1913.36 km


In [35]:
round(sum(distances))

32410

1 - 36990
2- 32410